<a href="https://colab.research.google.com/github/jallenrobern/CCMACLRL_EXERCISES_COM221ML/blob/main/Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1239]:
import pandas as pd
from sklearn.metrics import log_loss

## Dataset File

In [1240]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/train.csv?raw=true'
df = pd.read_csv(dataset_url)

In [1241]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [1242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [1243]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Mme', 'Mrs'])
df['Title'] = df['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Jonkeer', 'Dona', 'Sir', 'Capt', 'Don', 'Lady', 'Countess', 'Master', 'Miss'], 'Other')


df['AgeBin'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

df['FareBin'] = pd.qcut(df['Fare'], 4, labels=['Low', 'Mid', 'High', 'Very High'])

df.head()

<ipython-input-1243-006a27d24c46>:5: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['Title'] = df['Title'].replace(['Mme', 'Mrs'])


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title,AgeBin,FareBin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,Mr,Adult,Low
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Mr,Middle-aged,Very High
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Other,Adult,Mid
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Other,Adult,Very High
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Mr,Adult,Mid


In [1244]:
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

df['Embarked'].fillna('S', inplace=True)

df['Fare'].fillna(df['Fare'].median(), inplace=True)

In [1245]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title', 'AgeBin', 'FareBin'], drop_first=True)

In [1246]:
from sklearn.model_selection import train_test_split
X = df.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1247]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [1248]:
model.score(X_test, y_test)

0.8044692737430168

In [1249]:
from sklearn.metrics import confusion_matrix

y_predicted = model.predict(X_test)
confusion_matrix(y_test, y_predicted)

array([[90, 15],
       [20, 54]])

In [1250]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179



In [1251]:
from sklearn.metrics import log_loss

loss = log_loss(y_test, y_predicted)

In [1252]:
loss

7.047641724129053

## Test File

In [1253]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [1254]:
dt.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [1255]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [1256]:
dt['FamilySize'] = dt['SibSp'] + dt['Parch'] + 1

dt['Title'] = dt['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
dt['Title'] = dt['Title'].replace(['Mlle', 'Ms'], 'Miss')
dt['Title'] = dt['Title'].replace(['Mme', 'Mrs'])
dt['Title'] = dt['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Jonkeer', 'Dona', 'Sir', 'Capt', 'Don', 'Lady', 'Countess', 'Master'], 'Other')


dt['AgeBin'] = pd.cut(dt['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

dt['FareBin'] = pd.qcut(dt['Fare'], 4, labels=['Low', 'Mid', 'High', 'Very High'])

dt.head()

<ipython-input-1256-2fba7d92083a>:5: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  dt['Title'] = dt['Title'].replace(['Mme', 'Mrs'])


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title,AgeBin,FareBin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,Mr,Adult,Low
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,2,Mr,Middle-aged,Low
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,Mr,Senior,Mid
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,Mr,Adult,Mid
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,3,Mr,Adult,Mid


In [1257]:
dt['Age'] = dt.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

dt['Embarked'].fillna('S', inplace=True)

dt['Fare'].fillna(dt['Fare'].median(), inplace=True)

In [1258]:
dt = pd.get_dummies(dt, columns=['Sex', 'Embarked', 'Title', 'AgeBin', 'FareBin'], drop_first=True)

In [1259]:
dt.drop(columns=['Ticket', 'Cabin', 'Name', 'PassengerId'], inplace=True)

In [1260]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Pclass              418 non-null    int64  
 1   Age                 418 non-null    float64
 2   SibSp               418 non-null    int64  
 3   Parch               418 non-null    int64  
 4   Fare                418 non-null    float64
 5   FamilySize          418 non-null    int64  
 6   Sex_male            418 non-null    bool   
 7   Embarked_Q          418 non-null    bool   
 8   Embarked_S          418 non-null    bool   
 9   Title_Mr            418 non-null    bool   
 10  Title_Other         418 non-null    bool   
 11  AgeBin_Teenager     418 non-null    bool   
 12  AgeBin_Adult        418 non-null    bool   
 13  AgeBin_Middle-aged  418 non-null    bool   
 14  AgeBin_Senior       418 non-null    bool   
 15  FareBin_Mid         418 non-null    bool   
 16  FareBin_

## Sample Submission File

In [1261]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/f5fdfba1aabbbafcf2fe49ca6e1c8f92f69d1458/datasets/titanic/gender_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [1262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         891 non-null    int64  
 1   Survived            891 non-null    int64  
 2   Pclass              891 non-null    int64  
 3   Name                891 non-null    object 
 4   Age                 891 non-null    float64
 5   SibSp               891 non-null    int64  
 6   Parch               891 non-null    int64  
 7   Ticket              891 non-null    object 
 8   Fare                891 non-null    float64
 9   Cabin               204 non-null    object 
 10  FamilySize          891 non-null    int64  
 11  Sex_male            891 non-null    bool   
 12  Embarked_Q          891 non-null    bool   
 13  Embarked_S          891 non-null    bool   
 14  Title_Mr            891 non-null    bool   
 15  Title_Other         891 non-null    bool   
 16  AgeBin_T

In [1263]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Pclass              418 non-null    int64  
 1   Age                 418 non-null    float64
 2   SibSp               418 non-null    int64  
 3   Parch               418 non-null    int64  
 4   Fare                418 non-null    float64
 5   FamilySize          418 non-null    int64  
 6   Sex_male            418 non-null    bool   
 7   Embarked_Q          418 non-null    bool   
 8   Embarked_S          418 non-null    bool   
 9   Title_Mr            418 non-null    bool   
 10  Title_Other         418 non-null    bool   
 11  AgeBin_Teenager     418 non-null    bool   
 12  AgeBin_Adult        418 non-null    bool   
 13  AgeBin_Middle-aged  418 non-null    bool   
 14  AgeBin_Senior       418 non-null    bool   
 15  FareBin_Mid         418 non-null    bool   
 16  FareBin_

In [1264]:
sf.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [1265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         891 non-null    int64  
 1   Survived            891 non-null    int64  
 2   Pclass              891 non-null    int64  
 3   Name                891 non-null    object 
 4   Age                 891 non-null    float64
 5   SibSp               891 non-null    int64  
 6   Parch               891 non-null    int64  
 7   Ticket              891 non-null    object 
 8   Fare                891 non-null    float64
 9   Cabin               204 non-null    object 
 10  FamilySize          891 non-null    int64  
 11  Sex_male            891 non-null    bool   
 12  Embarked_Q          891 non-null    bool   
 13  Embarked_S          891 non-null    bool   
 14  Title_Mr            891 non-null    bool   
 15  Title_Other         891 non-null    bool   
 16  AgeBin_T

In [1266]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Pclass              418 non-null    int64  
 1   Age                 418 non-null    float64
 2   SibSp               418 non-null    int64  
 3   Parch               418 non-null    int64  
 4   Fare                418 non-null    float64
 5   FamilySize          418 non-null    int64  
 6   Sex_male            418 non-null    bool   
 7   Embarked_Q          418 non-null    bool   
 8   Embarked_S          418 non-null    bool   
 9   Title_Mr            418 non-null    bool   
 10  Title_Other         418 non-null    bool   
 11  AgeBin_Teenager     418 non-null    bool   
 12  AgeBin_Adult        418 non-null    bool   
 13  AgeBin_Middle-aged  418 non-null    bool   
 14  AgeBin_Senior       418 non-null    bool   
 15  FareBin_Mid         418 non-null    bool   
 16  FareBin_

In [1267]:
id = sf.pop('PassengerId')
y_pred = model.predict(dt)

submission_df = pd.DataFrame({
    'PassengerID': id,
    'Survived': y_pred
})

submission_df.to_csv('results.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
